In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import shutil


In [2]:
# Diretórios de origem (onde as imagens estão classificadas)
classificacao_dir = 'image/classificacao'
boa_classificacao_dir = os.path.join(classificacao_dir, 'boa')
ruim_classificacao_dir = os.path.join(classificacao_dir, 'ruim')

# Diretórios de destino (onde as imagens serão copiadas para treino)
treinamento_dir = 'image/treinamento'
boa_treinamento_dir = os.path.join(treinamento_dir, 'boa')
ruim_treinamento_dir = os.path.join(treinamento_dir, 'ruim')

# Caminho para salvar o modelo
modelo_path = 'model/modelo_solda.h5'


In [3]:
# Função para copiar as imagens para o destino correto
def copiar_imagens(origem_dir, destino_dir):
    # Verifica se o diretório de destino existe, se não, cria
    if not os.path.exists(destino_dir):
        os.makedirs(destino_dir)

    # Lista de imagens no diretório de origem
    imagens = [f for f in os.listdir(origem_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

    # Copiar as imagens para o diretório de destino
    for imagem in imagens:
        origem_imagem = os.path.join(origem_dir, imagem)
        destino_imagem = os.path.join(destino_dir, imagem)

        # Copiar a imagem se não estiver já no destino
        if not os.path.exists(destino_imagem):
            shutil.copy(origem_imagem, destino_imagem)
            print(f"Imagem {imagem} copiada para {destino_dir}")
        else:
            print(f"Imagem {imagem} já está em {destino_dir}, ignorando.")

# Copiar as imagens de "boa" e "ruim" para as pastas de treinamento
copiar_imagens(boa_classificacao_dir, boa_treinamento_dir)
copiar_imagens(ruim_classificacao_dir, ruim_treinamento_dir)

print("Imagens copiadas para as pastas de treinamento!")


Imagens copiadas para as pastas de treinamento!


In [4]:
# Gerador de imagens de treinamento com divisão para validação
train_datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    treinamento_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    treinamento_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse',
    subset='validation'
)


Found 19 images belonging to 2 classes.


Found 4 images belonging to 2 classes.


In [5]:
# Verifica se o modelo já existe, se existir, carrega o modelo
if os.path.exists(modelo_path):
    model = tf.keras.models.load_model(modelo_path)
    print("Modelo carregado, continuando o treinamento...")
else:
    # Caso contrário, cria e treina um novo modelo
    print("Treinando um novo modelo...")
    model = tf.keras.Sequential([
        # Camada convolucional 1
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        # Camada convolucional 2
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        # Camada densa
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')  # Para classificação "boa" e "ruim"
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Treinamento do modelo
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator  # Usando o gerador de validação
)


Treinando um novo modelo...


c:\Users\finan\OneDrive - Nimofast Brasil SA\Área de Trabalho\cod\python\AI\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\finan\OneDrive - Nimofast Brasil SA\Área de Trabalho\cod\python\AI\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.0000e+00 - loss: 0.7032 - val_accuracy: 0.5000 - val_loss: 0.3835
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.5263 - loss: 0.3634 - val_accuracy: 0.5000 - val_loss: 1.7951
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.4737 - loss: 1.8894 - val_accuracy: 1.0000 - val_loss: 0.0221
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.0218 - val_accuracy: 1.0000 - val_loss: 0.0374
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.0355 - val_accuracy: 1.0000 - val_loss: 0.1655
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 0.1568 - val_accuracy: 1.0000 - val_loss: 0.1155
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.1094 - val_accuracy: 1.0000 - val_loss: 0.0241
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 0.0228 - val_accuracy: 1.0000 - val_loss: 0.0077
Epoch 9/10
1

In [6]:
# Salvar o modelo treinado
model.save(modelo_path)
print(f"Modelo salvo em: {modelo_path}")


Modelo salvo em: model/modelo_solda.h5
